In [65]:
import pandas as pd
import os
import sys

# Ga drie niveaus omhoog vanuit de huidige submap
root_dir = os.path.abspath(os.path.join(os.getcwd(), "../../../"))
sys.path.append(root_dir)

# Importeer lokale modules
from preprocessing import (
    transformeer_woononderzoek_nederland,
    transformeer_leefbarometer_data,
    transformeer_cbs_data,
    laad_woningtekort_data,
    laad_data_invoerapplicatie
)
from api_scripts.api_utils import download_cbs_data

ImportError: cannot import name 'transformeer_woononderzoek_nederland' from 'preprocessing' (c:\Users\damoiseauxjmb\OneDrive - Provincie Limburg\Documenten\Python\indicatorenplan-limburg\teams\Leefbare_steden_en_dorpen\transformaties\preprocessing.py)

In [ ]:
# -----------------------------------------------------------------------------
# Instellingen en mappings
# -----------------------------------------------------------------------------
# Mapping van regio's
# Dit kan later verbeterd worden door alle buurt-wijk-gem-corop-prov mappings centraal op te slaan

regio_mapping = {
    "Nederland": "nl00",
    "Noord-Limburg": "cr37",
    "Midden-Limburg": "cr38",
    "Zuid-Limburg": "cr39",
}

In [ ]:
# -----------------------------------------------------------------------------
# Initialiseer de indicator dictionary
# -----------------------------------------------------------------------------
# Hierin worden indicatornamen gekoppeld aan hun corresponderende DataFrames

indicatoren_dict = {}

In [ ]:
# -----------------------------------------------------------------------------
# Indicator MO_11a: Woningtekort Data
# -----------------------------------------------------------------------------
print("Processing indicator: MO_11a")
df_mo_11a = laad_woningtekort_data(regio_mapping)

# Toevoegen aan de indicator dictionary
indicatoren_dict["MO_11a"] = df_mo_11a[["geoitem", "geolevel", "df_mo_11a", "period"]]

Processing indicator: MO_11a


c:\Users\damoiseauxjmb\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\damoiseauxjmb\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
# -----------------------------------------------------------------------------
# Indicator MO_11b: Woononderzoek Nederland en Limburg
# -----------------------------------------------------------------------------
print("Processing indicator: MO_11b")
df_woonderzoek_limburg = pd.read_csv("../../../data/Woononderzoek_nederland/Tevreden met woning - Corop-gebieden van Limburg.csv", sep=';')
df_woonderzoek_nederland = pd.read_csv("../../../data/Woononderzoek_nederland/Tevreden met woning - Nederland.csv", sep=';')

# Preprocessing
df_woonderzoek_limburg = transformeer_woononderzoek_nederland(df_woonderzoek_limburg, geolevel='corop_id')
df_woonderzoek_nederland = transformeer_woononderzoek_nederland(df_woonderzoek_nederland, geolevel='nederland')

# Combineer en verwerk 
df_mo_11b = pd.concat([df_woonderzoek_limburg, df_woonderzoek_nederland])

# # Toevoegen aan de indicator dictionary
indicatoren_dict["MO_11b"] = df_mo_11b

Processing indicator: MO_11b


,geoitem,MO_11b,period,dim_eigendom_1,dim_tevredenheid_0,geolevel
0,cr37,48,2009,2,zeer_tevreden,corop_id
1,cr38,44,2009,2,zeer_tevreden,corop_id
2,cr39,38,2009,2,zeer_tevreden,corop_id
3,cr37,22,2009,2,tevreden,corop_id
4,cr38,27,2009,2,tevreden,corop_id
...,...,...,...,...,...,...
70,nl00,3,2021,12,zeer_tevreden,nederland
71,nl00,5,2021,12,tevreden,nederland
72,nl00,2,2021,12,niet_tevreden_maar_ook_niet_ontevreden,nederland
73,nl00,1,2021,12,ontevreden,nederland


In [ ]:
# -----------------------------------------------------------------------------
# Indicator MO_10a, D_39a, D_39aa: Leefbarometer Data
# -----------------------------------------------------------------------------
print("Processing indicators: MO_10a, D_39a, D_39aa")
relevante_jaren = [2014, 2018, 2020, 2022]
column_renames = {
    "lbm": "MO_10a",
    "fys": "D_39a",
    "vrz": "D_39aa",
    "COROP_NAAM": "geoitem",
    "jaar": "period",
}

# Data verwerken
df_leefbarometer = transformeer_leefbarometer_data(
    input_bestand_pad="../../../data/Leefbarometer/Leefbaarometer-scores buurten 2002-2022.csv",
    bu_code_correcties_pad="../../../data/Buurtcodes/bu_code_correcties.xlsx",
    limburg_buurten_pad="../../../data/Buurtcodes/BU_WK_GM_codes.xls",
    relevante_jaren=relevante_jaren,
    column_renames=column_renames,
    regio_mapping=regio_mapping,
)

# Los toevoegen van indicatoren aan de dictionary
for indicator_code in ["MO_10a", "D_39a", "D_39aa"]:
    indicatoren_dict[indicator_code] = df_leefbarometer[["geoitem", "geolevel", "period", indicator_code]]

Processing indicators: MO_10a, D_39a, D_39aa


In [ ]:
# -----------------------------------------------------------------------------
# Indicator MO_12d: CBS Statline
# -----------------------------------------------------------------------------

# Download de CBS data: Limburgse COROP-regio's
df_mo_12d = download_cbs_data(
    table_code='70072NED', 
    geolevel='cr', 
    filter_limburg=True, 
    convert_to_geolevel_codes=True,
    keep_nl_data=True
)

# Transformeer de data
df_mo_12d = transformeer_cbs_data(df_mo_12d, geolevel='corop_id')

# Toevoegen aan de indicator dictionary
indicatoren_dict['MO_12d'] = df_mo_12d

Dataset met tabelcode '70072NED' succesvol opgehaald.
Nederland-rijen apart opgeslagen: 30 rijen.
RegioS opgeschoond.
Data gefilterd op geolevel 'cr': 1200 rijen over.
Geolevel-suffix '(cr)' verwijderd.
Data gefilterd op Limburgse COROP-regio's: 90 rijen over.
RegioS-namen vertaald naar codes.
Nederland-rijen toegevoegd aan de dataset: totaal 120 rijen.


In [ ]:
# -----------------------------------------------------------------------------
# Indicators R_118a, R_119a, R_120a, R_121a, R_122a, R_124a: Data Invoerapplicatie
# -----------------------------------------------------------------------------
print("Processing indicators: R_118a, R_119a, R_120a, R_121a, R_122a, R_124a")
df_invoerapplicatie = laad_data_invoerapplicatie(
    bron_bestand="../../../data/Invoerapplicatie/Invoerapplicatie Leefbare steden en dorpen.xlsx"
)

# Los toevoegen van indicatoren aan de dictionary
for indicator_code in ["R_118a", "R_119a", "R_120a", "R_121a", "R_122a", "R_124a"]:
    # Verwijder rijen met missende waarden in de geselecteerde kolommen
    filtered_df = df_invoerapplicatie[["geoitem", "geolevel", "period", indicator_code]].dropna()

    # Controleer of het gefilterde DataFrame niet leeg is
    if not filtered_df.empty:
        # Voeg het gefilterde DataFrame toe aan de dictionary
        indicatoren_dict[indicator_code] = filtered_df


Processing indicators: R_118a, R_119a, R_120a, R_121a, R_122a, R_124a


In [ ]:
# -----------------------------------------------------------------------------
# Controleer de inhoud van de dictionary
# -----------------------------------------------------------------------------
print("\nControle van indicator dictionary:")
for key, value in indicatoren_dict.items():
    if isinstance(value, pd.DataFrame):
        print(f"Indicator {key}, bevat een DataFrame met shape {value.shape}")
    else:
        print(f"Waarschuwing: Indicator {key} bevat geen geldig DataFrame")


Controle van indicator dictionary:
Indicator MO_11a, bevat een DataFrame met shape (21, 4)
Indicator MO_11b, bevat een DataFrame met shape (300, 6)
Indicator MO_10a, bevat een DataFrame met shape (12, 4)
Indicator D_39a, bevat een DataFrame met shape (12, 4)
Indicator D_39aa, bevat een DataFrame met shape (12, 4)
Indicator MO_12d, bevat een DataFrame met shape (120, 4)
Indicator R_118a, bevat een DataFrame met shape (1, 4)
Indicator R_119a, bevat een DataFrame met shape (1, 4)
Indicator R_120a, bevat een DataFrame met shape (1, 4)
Indicator R_121a, bevat een DataFrame met shape (1, 4)
Indicator R_122a, bevat een DataFrame met shape (1, 4)
Indicator R_124a, bevat een DataFrame met shape (1, 4)


In [ ]:
# -----------------------------------------------------------------------------
# Schrijf alle DataFrames weg naar de output folder
# -----------------------------------------------------------------------------
print("\nData wegschrijven naar bestanden...")
output_folder = "../../../Publicatie_bestanden/Leefbare_steden_en_dorpen"
for indicator, df in indicatoren_dict.items():
    bestandspad = os.path.join(output_folder, f"{indicator}.csv")
    df.to_csv(bestandspad, index=False)
    print(f"{indicator} opgeslagen als: {bestandspad}")


Data wegschrijven naar bestanden...
MO_11a opgeslagen als: ../../../Publicatie_bestanden/Leefbare_steden_en_dorpen\MO_11a.csv
MO_11b opgeslagen als: ../../../Publicatie_bestanden/Leefbare_steden_en_dorpen\MO_11b.csv
MO_10a opgeslagen als: ../../../Publicatie_bestanden/Leefbare_steden_en_dorpen\MO_10a.csv
D_39a opgeslagen als: ../../../Publicatie_bestanden/Leefbare_steden_en_dorpen\D_39a.csv
D_39aa opgeslagen als: ../../../Publicatie_bestanden/Leefbare_steden_en_dorpen\D_39aa.csv
MO_12d opgeslagen als: ../../../Publicatie_bestanden/Leefbare_steden_en_dorpen\MO_12d.csv
R_118a opgeslagen als: ../../../Publicatie_bestanden/Leefbare_steden_en_dorpen\R_118a.csv
R_119a opgeslagen als: ../../../Publicatie_bestanden/Leefbare_steden_en_dorpen\R_119a.csv
R_120a opgeslagen als: ../../../Publicatie_bestanden/Leefbare_steden_en_dorpen\R_120a.csv
R_121a opgeslagen als: ../../../Publicatie_bestanden/Leefbare_steden_en_dorpen\R_121a.csv
R_122a opgeslagen als: ../../../Publicatie_bestanden/Leefbare_ste